# RNNs in TensorFlow

## Agenda
* All about RNNs
* Implementation tricks & treats
* Live demo of Language Modeling

### 補助資料
http://qiita.com/t_Signull/items/21b82be280b46f467d1b

### From feed-forward to RNNs
feed-forward: 目標値(や推定誤差)から（結果を使用せずに）操作量を計算する。目標値から直接操作量を出すので、応答が早い一方で、操作量の計算中の誤差がそのまま計算に反映されてしまう。

* RNNはシーケンシャルなデータ(text, 遺伝子, 単語)の利点を得ることができる
* Directed cycle(有向閉路グラフ)
* すべてのステップはパラメータの総数を減らすために、weightをシェアする
* NLPの重要要素である
* 画像処理にも使用されることがある

### RNNsの問題点
*  long-termな依存関係をキャプチャするのは得意ではない
例えば、非常に長いシンボルが続いた後に、その後、最初のxまたはyが出現するという系列を学習する場合、最初の要素の情報を少なくともpステップ維持する機能をもつようにNNの重みを更新する必要がある。数十ステップなら可能であるが、数千ステップのような長期の系列は学習ができなかった。

そのため、、



### LSTMの流行
**Long Short Term Memory**

Long tem memory(長期記憶)とShort term memory(短期記憶)という神経科学における用語から取られている。
LSTMは **RNNの中間層のユニットをLSTM blockと呼ばれるメモリと3つのゲートを持つブロックに置き換える** ことで実現している

* 3つのゲート
    * Input Gate
    * Forget Gate
    * Output/Exposure Gate

In [1]:

import tensorflow as tf
import time
import os

DATA_PATH = './arvix_abstracts.txt'
HIDDEN_SIZE = 200
LR = 0.003
BATCH_SIZE = 64
NUM_STEPS = 50
SKIP_STEP = 40
LEN_GENERATED = 300
TEMPRATURE = 0.7

def create_rnn(seq, hidden_size=HIDDEN_SIZE):
    cell = tf.contrib.rnn.core_rnn_cell.GRUCell(hidden_size)
    in_state = tf.placeholder_with_default(
        cell.zero_state(tf.shape(seq)[0], tf.float32),[None, hidden_size])
    
    length = tf.reduce_sum(tf.reduce_max(tf.sign(seq), 2), 1)
    output, output_state = tf.nn.dynamic_rnn(cell, seq, length, in_state)
    return output, in_state, output_state

def create_model(seq, temp, vocab, hidden=HIDDEN_SIZE):
    seq = tf.one_hot(seq, len(vocab))
    output, in_state, out_state = create_rnn(seq, hidden)
    logits = tf.contrib.layers.fully_connected(output, len(vocab), None)
    loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=logits[:,:-1], labels=seq[:,1:]))
    
    sample = tf.multinomial(tf.exp(logits[:, -1] / temp), 1)[:, 0]
    return loss, sample, in_state, out_state

def vocab_encode(text, vocab):
    return [vocab.index(x) + 1 for x in text if x in vocab]

def vocab_decode(arr, vocab):
    return ''.join([vocab[x - 1] for x in arr])

def read_data(filename, vocab, window=NUM_STEPS, overlap=NUM_STEPS/2):
    for text in open(filename):
        text = vocab_encode(text, vocab)
        for start in range(0, len(text) - window, int(overlap)):
            chunk = text[start: start + window]
            chunk += [0] * (window - len(chunk))
            yield chunk
        

def read_batch(stream, batch_size=BATCH_SIZE):
    batch = []
    for element in stream:
        batch.append(element)
        if len(batch) == batch_size:
            yield batch
            batch = []
    yield batch
    

def online_interface(sess, vocab, seq, sample, temp, in_state, out_state, seed='T'):
    sentence = seed
    state = None
    for _ in range(LEN_GENERATED):
        batch = [vocab_encode(sentence[-1], vocab)]
        feed = {seq: batch, temp: TEMPRATURE}
        
        if state is not None:
            feed.update({in_state: state})
        index, state = sess.run([sample, out_state], feed)
        sentence += vocab_decode(index, vocab)
    print(sentence)
    
def trainning(vocab, seq, loss, optimizer, global_step, temp, sample, in_state, out_state):
    saver = tf.train.Saver()
    start = time.time()
    
    with tf.Session() as sess:
        writer = tf.summary.FileWriter('graphs/gist', sess.graph)
        sess.run(tf.global_variables_initializer())
        
        ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/arvix/checkpoint'))
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
        
        iteration = global_step.eval()
        for batch in read_batch(read_data(DATA_PATH, vocab)):
            batch_loss, _ = sess.run([loss, optimizer], {seq: batch})
            if (iteration + 1) % SKIP_STEP == 0:
                print('Iter {}. \n    Loss {}. Time {}'.format(iteration, batch_loss, time.time() - start))
                online_interface(sess, vocab, seq, sample, temp, in_state, out_state)
                start = time.time()
                
                if not os.path.exists('checkpoints/arvix/char-rnn'): os.makedirs('checkpoints/arvix/char-rnn')
                
                saver.save(sess, 'checkpoints/arvix/char-rnn', iteration)
            iteration += 1
    

def main():
    vocab = (
            " $%'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            "\\^_abcdefghijklmnopqrstuvwxyz{|}")
    
    seq = tf.placeholder(tf.int32, [None, None])
    temp = tf.placeholder(tf.float32)
    loss, sample, in_state, out_state = create_model(seq, temp, vocab)
    global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='globa_step')
    optimizer = tf.train.AdamOptimizer(LR).minimize(loss, global_step=global_step)
    trainning(vocab, seq, loss, optimizer, global_step, temp, sample, in_state, out_state)

if __name__ == '__main__':
    main()

Iter 39. 
    Loss 9486.994140625. Time 10.257993936538696
T=ii  e e e  e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e 
Iter 79. 
    Loss 8290.7099609375. Time 9.71593713760376
Twon the ter the ter the ter te ter te ter te ter te ter te ter te ter te ter te ter te ter te ter te ter an the ter an the ter te ter te ter an the ter an the ter an the ter an the ter an the ter an the ter an the ter an the ter an the ter an the ter an the ter an the ter an the ter an the ter an th
Iter 119. 
    Loss 7533.07958984375. Time 9.670512199401855
The the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the th

## Neural Language Modeling
* センテンスがどの程度似ているかを測定できる
* 機械翻訳にとって重要なinputである
* 新規の文書を作成できる

### 主なアプローチ
* 単語レベル(n-gram)
    * 伝統的なやり方
    * 前のn-gramにもとづいて次のwordを予測するためにモデルを学習する
    * 多くの語彙を必要とする
    * Out Of Vocabularyが発生しない
    * 大量のメモリが必要
* 文字レベル
    * 入力も出力も文字である
    *  利点
        * 語彙が少なくても良い
        *  word embeddings が不要
        * 訓練が高速
    * 欠点
        * 言葉がちんぷんかんぷんなものができる
    * Unknownなものだけ文字レベルに切り替える方法もある
* subword(音声)レベル
    * inputもoutputもsubwords
    * W を最も頻出する単語とする
    * S を最も頻出する音節とする
    * それらを文字に分割する
    * wordレベルやcharactorレベルのモデルよりもよく動いているように見える